In [2]:
from dotenv import load_dotenv
load_dotenv() 

import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

c:\Users\chanc\anaconda3\envs\cityB\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
feature_store = get_feature_store()

2025-05-10 10:48:22,156 INFO: Initializing external client
2025-05-10 10:48:22,158 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 10:48:23,464 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1230996


In [5]:
from datetime import datetime, timedelta
import pandas as pd  


current_date = pd.Timestamp.now()
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
# Ensure fetch_data_from and fetch_data_to are timezone-aware to match ts_data.pickup_hour
fetch_data_from = fetch_data_from.tz_localize(None)
fetch_data_to = fetch_data_to.tz_localize(None)
ts_data.pickup_hour = ts_data.pickup_hour.dt.tz_localize(None)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-05-10 10:48:27,299 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 10:48:27,309 INFO: Initializing external client
2025-05-10 10:48:27,310 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 10:48:28,043 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1230996
Fetching data from 2025-04-11 10:48:27.298314 to 2025-05-10 09:48:27.298314


AttributeError: 'NoneType' object has no attribute 'get_batch_data'

In [ ]:
ts_data.sort_values(["station_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,station_id,rides
0,2025-04-12 06:00:00,HB101,2
1,2025-04-12 12:00:00,HB101,16
2,2025-04-12 18:00:00,HB101,18
3,2025-04-13 00:00:00,HB101,10
4,2025-04-13 06:00:00,HB101,13
...,...,...,...
331,2025-05-09 00:00:00,HB305,1
332,2025-05-09 06:00:00,HB305,11
333,2025-05-09 12:00:00,HB305,28
334,2025-05-09 18:00:00,HB305,3


In [51]:
ts_data

,pickup_hour,station_id,rides
0,2025-05-02 12:00:00,HB105,32
1,2025-04-25 00:00:00,HB105,4
2,2025-04-21 18:00:00,HB101,31
3,2025-04-19 06:00:00,HB101,2
4,2025-05-09 06:00:00,HB101,7
...,...,...,...
331,2025-04-17 12:00:00,HB105,20
332,2025-05-08 18:00:00,HB101,23
333,2025-05-02 06:00:00,HB305,19
334,2025-05-07 06:00:00,HB105,17


In [1]:
from src.data_utils import transform_ts_data_info_features_and_target

features , _ = transform_ts_data_info_features_and_target(ts_data, window_size=28, step_size=23)

NameError: name 'ts_data' is not defined